In [10]:
import pandas as pd
import nltk
import os
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import warnings
warnings.filterwarnings('ignore')
nltk.download('stopwords')

In [12]:
os.chdir('E:/UCL/BIR22/Sentiment analysis')

# 1. Importation et preparation

## 1.1. Dataset d'entrainement

### 1.1.1. Creation des datasets

In [13]:
#Importation

Data_entrainement = pd.read_csv("./training.1600000.processed.noemoticon.csv", header = None, encoding = "ISO-8859-1")

In [14]:
#On donne des noms aux colonnes

Data_entrainement.columns = ['Polarity', 'Tweet_ID','Date','Query_username','Username','Tweet_text']
Data_entrainement

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [15]:
#montre qu'il n'y a bien que 2 polarité dans le tableau
a = []
for i in Data[0]:
    if i not in a:
        a.append(i)
print(a)
#0 = negative, 4 = positive

[0, 4]


In [19]:
#Selection de tweets

Datapos = Data_entrainement[Data_entrainement['Polarity']==4]
Dataneg = Data_entrainement[Data_entrainement['Polarity']==0]

Data_sub = pd.concat([Datapos.sample(5000),Dataneg.sample(5000)],axis=0)
Data_sub.head()

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
908271,4,1696135115,Mon May 04 07:40:55 PDT 2009,NO_QUERY,mikesawriter,@MmmBaileys @carli_chick Hello
895174,4,1692567464,Sun May 03 20:16:43 PDT 2009,NO_QUERY,CodyOW,@DSCarey tell me that didn't make your day hap...
1551520,4,2184090411,Mon Jun 15 15:24:29 PDT 2009,NO_QUERY,chrisyoungken,watching big brother. LOL @ what they did to s...
1392222,4,2053389604,Sat Jun 06 04:22:52 PDT 2009,NO_QUERY,lorna_hughes,"@cromwellswirral sounds good, will make sure I..."
1240101,4,1993705920,Mon Jun 01 10:48:13 PDT 2009,NO_QUERY,VisitBooneNC,@redmantw I love the Moses Cone Manor It's g...


In [20]:
#pour vérifier que pos et neg ont la même longueur
print(len(Datapos),len(Dataneg))

800000 800000


### 1.1.2. Preprocessing

In [28]:
def prepro(text,i):
    clean=text[i].astype(str).str.replace("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ")
    lowercase=clean.str.lower()
    words=lowercase.apply(word_tokenize)
    stop_words = stopwords.words('english') 
    filtered_words = words.apply(lambda x:[word for word in x if word not in stop_words])
    porter = PorterStemmer()
    text['Tweet_clean']=filtered_words.apply(lambda x:[porter.stem(word) for word in x])
    return text

In [51]:
Data_sub_clean = prepro(Data_sub,'Tweet_text')

In [53]:
#reset l'index

Data_sub_clean.reset_index(inplace=True)
del Data_sub_clean['index']

In [54]:
Data_sub_clean.head()

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text,Tweet_clean
0,4,1696135115,Mon May 04 07:40:55 PDT 2009,NO_QUERY,mikesawriter,@MmmBaileys @carli_chick Hello,"[chick, hello]"
1,4,1692567464,Sun May 03 20:16:43 PDT 2009,NO_QUERY,CodyOW,@DSCarey tell me that didn't make your day hap...,"[tell, make, day, happier, though, lt, 3]"
2,4,2184090411,Mon Jun 15 15:24:29 PDT 2009,NO_QUERY,chrisyoungken,watching big brother. LOL @ what they did to s...,"[watch, big, brother, lol, sree, still, done, ..."
3,4,2053389604,Sat Jun 06 04:22:52 PDT 2009,NO_QUERY,lorna_hughes,"@cromwellswirral sounds good, will make sure I...","[sound, good, make, sure, call, next, time, irbi]"
4,4,1993705920,Mon Jun 01 10:48:13 PDT 2009,NO_QUERY,VisitBooneNC,@redmantw I love the Moses Cone Manor It's g...,"[love, mose, cone, manor, got, soul]"


### 1.1.3. Separation en data d'entrainement et de test

In [55]:
#Seperation en data d'entrainement et de test

Proportion_training = 0.7
Data_train, Data_test = train_test_split(Data_sub_clean, test_size=(1-Proportion_training))

In [56]:
print(len(Data_train),len(Data_test))
Data_train.head()

6999 3001


,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text,Tweet_clean
4579,4,2044147290,Fri Jun 05 08:56:20 PDT 2009,NO_QUERY,Winstonita,#followfriday @TonyLettws @Whatever-ista @reed...,"[followfriday, ista, love]"
6874,0,2198659794,Tue Jun 16 16:23:20 PDT 2009,NO_QUERY,CBeSOS,@MissAliH everyones gones to bed so i'm offici...,"[everyon, gone, bed, offici, sat, tod, livingr..."
7828,0,1994201994,Mon Jun 01 11:34:14 PDT 2009,NO_QUERY,crossfitchat,Meeting ran over so I couldn't make Crossfit. ...,"[meet, ran, make, crossfit, guess, miss, mke, ..."
7112,0,2008020472,Tue Jun 02 13:30:09 PDT 2009,NO_QUERY,keren4562,@Dannymcfly Danny why McFLY are'nt coming to i...,"[danni, mcfli, nt, come, israel, hate, us, plz..."
7695,0,2011565617,Tue Jun 02 19:13:09 PDT 2009,NO_QUERY,Eminemdrdre00,@EndlessDennis Well she wont be in a match *L...,"[well, wont, match, layla, candic, michel, amp..."


In [130]:
Data_train.reset_index(inplace=True)
del Data_train['index']

Data_test.reset_index(inplace=True)
del Data_test['index']

### 1.1.4. Creation de la liste de features

In [106]:
all_words = []

for t in Data_sub_clean["Tweet_clean"]:
    for w in t:
        all_words.append(w)

freq_all_words = nltk.FreqDist(all_words)


list_features = [freq_all_words.most_common(3000)[i][0] for i in range(3000)] #la liste des features 
    #reprend les 3000 mots les plus fréquents


### 1.1.5. Transformation des tweets en dictionnaires indiquant la présence de features

In [122]:
#fonction qui permettra de transformer les tweets en dictionnaires indiquant la présence des features

def find_features(document):
    words = set(document)
    features = {}
    for w in list_features:
        features[w] = (w in words)

    return features

In [131]:
#Definition du training set

training_set = [(find_features(Data_train["Tweet_clean"][index]),Data_train["Polarity"][index]) for index in range(len(Data_train["Tweet_clean"]))]

In [143]:
len(training_set)

6999

In [142]:
#Définition du testing set

testing_set = [(find_features(Data_test["Tweet_clean"][index]),Data_test["Polarity"][index]) for index in range(len(Data_test["Tweet_clean"]))]

In [144]:
len(testing_set)

3001

## 1.2. Data à analyser

In [151]:
#Importation

dfDonaldTrump = pd.read_csv('hashtag_donaldtrump.csv', delimiter=',', engine = 'python')
dfJoeBiden = pd.read_csv('hashtag_joebiden.csv', delimiter=',')

In [172]:
dfDT_small = pd.DataFrame({"tweet":dfDonaldTrump["tweet"]})
dfJB_small = pd.DataFrame({"tweet":dfJoeBieden["tweet"]})

### 1.2.1. Preprocessing

In [174]:
dfDT_clean = prepro(dfDT_small,'tweet')

In [ ]:
dfJB_clean = prepro(dfJB_small,'tweet')

In [175]:
dfDT_clean.head()

,tweet,Tweet_clean
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,"[elecciones2020, en, florida, joebiden, dice, ..."
1,"Usa 2020, Trump contro Facebook e Twitter: cop...","[usa, 2020, trump, contro, facebook, e, twitte..."
2,"#Trump: As a student I used to hear for years,...","[trump, student, use, hear, year, ten, year, h..."
3,2 hours since last tweet from #Trump! Maybe he...,"[2, hour, sinc, last, tweet, trump, mayb, busi..."
4,You get a tie! And you get a tie! #Trump â€˜s ...,"[get, tie, get, tie, trump, ralli, iowa]"


### 1.2.2. Transformations des tweets en dictionnaires indicants la présence des features

In [176]:
tweet_set_DT = [(find_features(dfDT_clean["Tweet_clean"][index])) for index in range(len(dfDT_clean["Tweet_clean"]))]

MemoryError: 

In [ ]:
tweet_set_JB = [(find_features(dfJB_clean["Tweet_clean"][index])) for index in range(len(dfJB_clean["Tweet_clean"]))]

# Sentiment analysis

## Entrainement

In [145]:
#train the algo
classifier = nltk.NaiveBayesClassifier.train(training_set)

## Test

In [146]:
#test the algo
test = nltk.classify.accuracy(classifier, testing_set)

In [147]:
print("L'accurancy est de {} %".format(test*100))

L'accurancy est de 69.71009663445517 %


In [149]:
classifier.show_most_informative_features(15)

Most Informative Features
                     ugh = True                0 : 4      =     35.2 : 1.0
                     sad = True                0 : 4      =     32.3 : 1.0
                  welcom = True                4 : 0      =     21.6 : 1.0
                    lost = True                0 : 4      =     20.3 : 1.0
                    alon = True                0 : 4      =     13.1 : 1.0
                internet = True                0 : 4      =     11.7 : 1.0
                unfortun = True                0 : 4      =     11.1 : 1.0
                  broken = True                0 : 4      =     10.4 : 1.0
                     due = True                0 : 4      =     10.4 : 1.0
                     www = True                4 : 0      =      9.8 : 1.0
                 stomach = True                0 : 4      =      9.0 : 1.0
                     cri = True                0 : 4      =      9.0 : 1.0
                    fail = True                0 : 4      =      8.5 : 1.0

## Utilisation

In [ ]:
#Use the algo sur le set de Donald Trump
classifier.classify(tweet_set_DT)

In [ ]:
#Use the algo sur le set de Joe Biden
classifier.classify(tweet_set_JB)